<a href="https://colab.research.google.com/github/gopalkalpande/Detecting-duplicate-product-listings-in-an-e-commerce-inventory/blob/master/technosolution_data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")


import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

from sklearn.model_selection import train_test_split
from sklearn import model_selection

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn.externals import joblib

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!ls "/content/drive/My Drive/data/technosolution/"

'df1.productId[4000]+'\''.jpeg'\'''
 Machinelearning_Assignment
 Machinelearning_Assignment-20190504T163245Z-001.zip
 preprocessed.csv


In [0]:
!wget --header="Host: doc-00-90-docs.googleusercontent.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3" --header="Accept-Language: en-US,en;q=0.9,te;q=0.8" --header="Referer: https://drive.google.com/drive/folders/1fR0KHk5nJpXxFbV5ls24CrtB3uA0hGxv" --header="Cookie: AUTH_cnju7gldtplg5tnpmc7is4aopd1g9b2q_nonce=bmkmg0g6j8na6; _ga=GA1.2.259530805.1548430358; NID=164=Q1l4greqojflMYT-hZ7k68gzR8rRy8IFA6d2mq57LrRjeQFIhs_x_5NPz2LFHglaBC9JEPcxprX9x2ycvosUPDxhcHWbG07gtXH1Fxx9YHa3Sy7V0uuYAI2EWYre2_ZJ7Vyx7Dg7auUMy8h-Amar7uoEj7QKHeg2eBpwvTxHSiM" --header="Connection: keep-alive" "https://doc-00-90-docs.googleusercontent.com/docs/securesc/kqa3c7jgtesc23675e7rrupvha80q0dp/k5ubkmq41uqdsn0li8uqorf362m9lq3r/1557064800000/13072423780617688646/14104312713827783075/1ULGDoH0b_U3UBNEFjP7fjM_RBFRC3tiF?e=download&nonce=bmkmg0g6j8na6&user=14104312713827783075&hash=e2edbo1tis3057hp6tii3fjune3tlrdl" -O "Machinelearning_Assignment-20190504T163245Z-001.zip" -c

--2019-05-05 16:23:36--  https://doc-00-90-docs.googleusercontent.com/docs/securesc/kqa3c7jgtesc23675e7rrupvha80q0dp/k5ubkmq41uqdsn0li8uqorf362m9lq3r/1557064800000/13072423780617688646/14104312713827783075/1ULGDoH0b_U3UBNEFjP7fjM_RBFRC3tiF?e=download&nonce=bmkmg0g6j8na6&user=14104312713827783075&hash=e2edbo1tis3057hp6tii3fjune3tlrdl
Resolving doc-00-90-docs.googleusercontent.com (doc-00-90-docs.googleusercontent.com)... 108.177.127.132, 2a00:1450:4013:c07::84
Connecting to doc-00-90-docs.googleusercontent.com (doc-00-90-docs.googleusercontent.com)|108.177.127.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-zip-compressed]
Saving to: ‘Machinelearning_Assignment-20190504T163245Z-001.zip’

Machinelearning_Ass     [       <=>          ]  67.07M  11.4MB/s    in 5.9s    

2019-05-05 16:23:43 (11.4 MB/s) - ‘Machinelearning_Assignment-20190504T163245Z-001.zip’ saved [70328665]



In [0]:
!unzip Machinelearning_Assignment-20190504T163245Z-001.zip -d "/content/drive/My Drive/data/technosolution/"

Archive:  Machinelearning_Assignment-20190504T163245Z-001.zip
  inflating: /content/drive/My Drive/data/technosolution/Machinelearning_Assignment/sample.txt  
  inflating: /content/drive/My Drive/data/technosolution/Machinelearning_Assignment/Assignment _ Problem Statement.docx  
  inflating: /content/drive/My Drive/data/technosolution/Machinelearning_Assignment/Readme.pdf  
  inflating: /content/drive/My Drive/data/technosolution/Machinelearning_Assignment/datafile.csv  


In [0]:
!ls "/content/drive/My Drive/data/technosolution/"

Machinelearning_Assignment  Machinelearning_Assignment-20190504T163245Z-001.zip


In [0]:
!ls "/content/drive/My Drive/data/technosolution/Machinelearning_Assignment"

'Assignment _ Problem Statement.docx'   datafile.csv   Readme.pdf   sample.txt


In [0]:
df = pd.read_csv("/content/drive/My Drive/data/technosolution/Machinelearning_Assignment/datafile.csv")

In [0]:
df.head(2)

,productId,title,description,imageUrlStr,mrp,sellingPrice,specialPrice,productUrl,categories,productBrand,...,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52
0,TOPE9ABBZU3HZRHN,Citrine Casual Short Sleeve Printed Women's Pi...,This beautiful printed modal top from Citrine ...,http://img.fkcdn.com/image/top/r/h/n/1-1-wwtpw...,1099,329.0,329.0,http://dl.flipkart.com/dl/citrine-casual-short...,"Apparels>Women>Western Wear>Shirts, Tops & Tun...",Citrine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TOPE9ABBBTJYDSQE,Citrine Casual Short Sleeve Printed Women's Pi...,This beautiful printed modal top from Citrine ...,http://img.fkcdn.com/image/top/r/h/n/1-1-wwtpw...,1099,329.0,329.0,http://dl.flipkart.com/dl/citrine-casual-short...,"Apparels>Women>Western Wear>Shirts, Tops & Tun...",Citrine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
df.columns

Index(['productId', 'title', 'description', 'imageUrlStr', 'mrp',
       'sellingPrice', 'specialPrice', 'productUrl', 'categories',
       'productBrand', 'productFamily', 'inStock', 'codAvailable', 'offers',
       'discount', 'shippingCharges', 'deliveryTime', 'size', 'color',
       'sizeUnit', 'storage', 'displaySize', 'keySpecsStr', 'detailedSpecsStr',
       'specificationList', 'sellerName', 'sellerAverageRating',
       'sellerNoOfRatings', 'sellerNoOfReviews', 'sleeve', 'neck', 'idealFor',
       'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35',
       'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39',
       'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43',
       'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47',
       'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51',
       'Unnamed: 52'],
      dtype='object')

In [0]:
df.shape

(346902, 53)

In [0]:
#Sorting data according to ProductId in ascending order
df1=df.sort_values('productId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [0]:
df1.productId.nunique()

346813

In [0]:
#Deduplication of entries
df1 = df1.drop_duplicates(subset={'productId'}, keep='first', inplace=False)
df1.shape

(346813, 53)

In [0]:
df1.imageUrlStr.nunique()

87815

In [0]:
#Sorting data according to imageUrlStr in ascending order
df1=df1.sort_values('imageUrlStr', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [0]:
#Deduplication of entries
df1 = df1.drop_duplicates(subset={'imageUrlStr'}, keep='first', inplace=False)
df1.shape

(87815, 53)

In [0]:
df1.productUrl.nunique()

87815

In [0]:
df1.inStock.value_counts()

True     43981
False    43834
Name: inStock, dtype: int64

## As I'm using Google Colab and there are restricted resources; I will try to solve only for the in stock products and going to remove out of stock products.

In [0]:
df1 = df1[df1.inStock != False]

In [0]:
df1.inStock.value_counts()

True    43981
Name: inStock, dtype: int64

In [0]:
df1.shape

(43981, 53)

#Lets work on NA Values

In [0]:
df1.isna().any()

productId              False
title                   True
description             True
imageUrlStr            False
mrp                    False
sellingPrice            True
specialPrice            True
productUrl             False
categories             False
productBrand           False
productFamily           True
inStock                False
codAvailable           False
offers                  True
discount               False
shippingCharges         True
deliveryTime            True
size                   False
color                   True
sizeUnit                True
storage                 True
displaySize             True
keySpecsStr             True
detailedSpecsStr        True
specificationList       True
sellerName              True
sellerAverageRating    False
sellerNoOfRatings       True
sellerNoOfReviews       True
sleeve                  True
neck                    True
idealFor                True
Unnamed: 32             True
Unnamed: 33             True
Unnamed: 34   

In [0]:
df1 = df1.dropna(axis=1, how='any')

In [0]:
df1.isna().any()

productId              False
imageUrlStr            False
mrp                    False
productUrl             False
categories             False
productBrand           False
inStock                False
codAvailable           False
discount               False
size                   False
sellerAverageRating    False
dtype: bool

In [0]:
# as I consider only in stock data inStock column is irrelevent.
# also the columns other than productId, imageUrlStr and productUrl appears to be unrelevent for detecting duplicasy.
# so......remove them all.

df1 = df1.drop(labels=['mrp', 'categories', 'productBrand', 'inStock', 'codAvailable', 'discount', 'size', 'sellerAverageRating'], axis=1)
df1.head()

,productId,imageUrlStr,productUrl
155830,TOPEG4YDVT9WX6FB,http://img.fkcdn.com/image/apparels-combo/2/f/...,http://dl.flipkart.com/dl/ruhaan-s-casual-slee...
157596,TOPEDBEXXFA89JWG,http://img.fkcdn.com/image/apparels-combo/3/s/...,http://dl.flipkart.com/dl/vama-casual-full-sle...
245150,TOPEGSUWFUUGEXQU,http://img.fkcdn.com/image/apparels-combo/6/r/...,http://dl.flipkart.com/dl/skidlers-casual-shor...
195773,TOPEG4YDFJUTPGZD,http://img.fkcdn.com/image/apparels-combo/b/a/...,http://dl.flipkart.com/dl/ruhaan-s-casual-slee...
236377,TOPEAZU6K7UGGHYH,http://img.fkcdn.com/image/apparels-combo/c/g/...,http://dl.flipkart.com/dl/ruhaan-s-casual-full...


In [0]:
df1.nunique()

productId      43981
imageUrlStr    43981
productUrl     43981
dtype: int64

In [0]:
df1.shape

(43981, 3)

# Now as the rows contains only unique information, I will move forward to data extraction in the next file.